In [41]:
import geopandas as gpd
import json
from shapely.ops import nearest_points
import pandas as pd
import fiona
from shapely.geometry import shape,mapping, Point, Polygon, MultiPolygon
from shapely.ops import cascaded_union

In [2]:
admin2 = gpd.read_file('gadm36_ETH_shp/gadm36_ETH_3.shp')
admin2['Gadm Name 2'] = admin2['NAME_2']

In [3]:
pts3 = admin2.centroid.unary_union

In [4]:
from shapely.ops import nearest_points
# unary union of the gpd2 geomtries 

def near(point, pts=pts3):
     # find the nearest point and return the corresponding Place value
    nearest = admin2.centroid == nearest_points(point, pts)[1]
    return admin2[nearest]['Gadm Name 2'].get_values()[0]

In [5]:
maln = gpd.read_file('malnutrition_eth.geojson' )
pop = gpd.read_file("population_ETH_2018.geojson")
travel = gpd.read_file("travel_time.geojson")

In [6]:
flood = pd.read_csv('flood-model.csv')
flood = gpd.GeoDataFrame(
    flood, geometry=gpd.points_from_xy(flood.longitude, flood.latitude))

In [7]:
chirps = pd.read_csv('CHIRPS-combined.csv')

chirps = gpd.GeoDataFrame(
    chirps, geometry=gpd.points_from_xy(chirps.longitude, chirps.latitude))

In [8]:
pop = pop[pop['Population']>0]

In [9]:
maln['centroid']=maln.geometry.apply(lambda x: x.centroid)
pop['centroid']=pop.geometry.apply(lambda x: x.centroid)
admin2['centroid']=admin2.geometry.apply(lambda x: x.centroid)

In [10]:
pop['Gadm Name 2'] = pop.apply(lambda row: near(row.centroid), axis=1)
maln['Gadm Name 2'] = maln.apply(lambda row: near(row.centroid), axis=1)

# for Flood and CHIRPS, just use geometry, not centroid since these are just points not shapes
flood['Gadm Name 2'] = flood.apply(lambda row: near(row.geometry), axis=1)
chirps['Gadm Name 2'] = chirps.apply(lambda row: near(row.geometry), axis=1)

In [11]:
del(pop['centroid'])
del(maln['centroid'])

In [12]:
pop.to_file("pop_ETH.geojson", driver="GeoJSON")
maln.to_file("maln_ETH.geojson", driver="GeoJSON")
flood.to_file("flood_ETH.geojson", driver="GeoJSON")
chirps.to_file("CHIRPS_combined.geojson", driver="GeoJSON")

### Post processing CHIRPS to handle data outside Ethiopia

In [36]:
c = gpd.read_file("CHIRPS_combined.geojson")

Here we read in the Ethiopia shape, merge it to one large polygon, and detect points which are within Ethiopia.

We have to do this because our prior processing mapped a GADM 2 admin to every point, even those in South Sudan.

In [58]:
polys = []
multipol = fiona.open("gadm36_ETH_shp/gadm36_ETH_1.shp")
for n in multipol:
    polys.append(Polygon(n['geometry']['coordinates'][0]))
    
u = cascaded_union(polys)

In [61]:
c['Ethiopia'] = c.geometry.apply(lambda x: x.within(u))

In [62]:
c.groupby(['Ethiopia']).geometry.count()

Ethiopia
False    349067
True     117668
Name: geometry, dtype: int64

In [64]:
def get_gadm(row): 
    if row.Ethiopia == True:
        return row['Gadm Name 2']
    else:
        return None

In [67]:
c['Gadm ETH'] = c.apply(lambda x: get_gadm(x), axis=1)

In [71]:
c['Gadm Name 2'] = c['Gadm ETH']

In [72]:
c.to_file("CHIRPS_combined.geojson", driver="GeoJSON")